# Day 09. Exercise 00
# Regularization

## 0. Imports

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score as cv
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
import pickle

## 1. Preprocessing

1. Read the file `dayofweek.csv` that you used in the previous day to a dataframe.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [2]:
df = pd.read_csv("../data/dayofweek.csv")
df

,Unnamed: 0,numTrials,hour,dayofweek,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,...,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1
0,0,-0.788667,-2.562352,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,-0.756764,-2.562352,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,-0.724861,-2.562352,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3,-0.692958,-2.562352,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4,-0.661055,-2.562352,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,1681,-0.533442,0.945382,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1682,1682,-0.629151,0.945382,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1683,1683,-0.597248,0.945382,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1684,1684,-0.565345,0.945382,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
X = df.drop('dayofweek', axis=1)
y = df['dayofweek']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## 2. Logreg regularization

### a. Default regularization

1. Train a baseline model with the only parameters `random_state=21`, `fit_intercept=False`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model


The result of the code where you trained and evaluated the baseline model should be exactly like this (use `%%time` to get the info about how long it took to run the cell):

```
train -  0.62902   |   valid -  0.59259
train -  0.64633   |   valid -  0.62963
train -  0.63479   |   valid -  0.56296
train -  0.65622   |   valid -  0.61481
train -  0.63397   |   valid -  0.57778
train -  0.64056   |   valid -  0.59259
train -  0.64138   |   valid -  0.65926
train -  0.65952   |   valid -  0.56296
train -  0.64333   |   valid -  0.59701
train -  0.63674   |   valid -  0.62687
Average accuracy on crossval is 0.60165
Std is 0.02943
```

In [6]:
def print_score(input_model):
  s = []
  for train, valid in cv.split(X_train, y_train):
    input_model.fit(X_train.iloc[train], y_train.iloc[train])
    train_pred = input_model.predict(X_train.iloc[train])
    train = accuracy_score(y_train.iloc[train], train_pred)
    valid_pred = input_model.predict(X_train.iloc[valid])
    valids = accuracy_score(y_train.iloc[valid], valid_pred)
    s.append(valids)
    print("train - %.5f | " %train, end='')
    print("valid - %.5f " %valids)

  print('Average accuracy on crossval is  %.5f ' %np.mean(s))
  print('Std is %.5f ' %np.std(s))

In [7]:
%%time
log_reg = LogisticRegression(fit_intercept=False, random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(log_reg)

/Users/hbrent/PythonDS/MINE/myenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/Users/hbrent/PythonDS/MINE/myenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/

train - 0.54081 | valid - 0.47407 
train - 0.55317 | valid - 0.54815 
train - 0.56554 | valid - 0.51111 
train - 0.59027 | valid - 0.60000 


/Users/hbrent/PythonDS/MINE/myenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/Users/hbrent/PythonDS/MINE/myenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/

train - 0.57049 | valid - 0.57037 
train - 0.56884 | valid - 0.55556 
train - 0.58368 | valid - 0.60741 
train - 0.58533 | valid - 0.47407 
train - 0.53624 | valid - 0.51493 
train - 0.56919 | valid - 0.56716 
Average accuracy on crossval is  0.54228 
Std is 0.04496 
CPU times: user 2.01 s, sys: 1.22 s, total: 3.23 s
Wall time: 686 ms


/Users/hbrent/PythonDS/MINE/myenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/Users/hbrent/PythonDS/MINE/myenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/

### b. Optimizing regularization parameters

1. In the cells below try different values of penalty: `none`, `l1`, `l2` – you can change the values of solver too.

In [8]:
%%time
log_reg = LogisticRegression(fit_intercept=False, random_state=21, penalty='l1', solver = 'liblinear')
cv = StratifiedKFold(n_splits=10)
print_score(log_reg)

train - 0.64551 | valid - 0.62222 
train - 0.65540 | valid - 0.63704 
train - 0.64138 | valid - 0.58519 
train - 0.64880 | valid - 0.62222 
train - 0.64798 | valid - 0.60741 
train - 0.64880 | valid - 0.60741 
train - 0.64880 | valid - 0.69630 
train - 0.66777 | valid - 0.58519 
train - 0.64662 | valid - 0.59701 
train - 0.63921 | valid - 0.59701 
Average accuracy on crossval is  0.61570 
Std is 0.03120 
CPU times: user 610 ms, sys: 371 ms, total: 982 ms
Wall time: 229 ms


In [9]:
%%time
log_reg = LogisticRegression(fit_intercept=False, random_state=21, penalty='none', solver='newton-cg')
cv = StratifiedKFold(n_splits=10)
print_score(log_reg)

/Users/hbrent/PythonDS/MINE/myenv/lib/python3.7/site-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,


train - 0.69909 | valid - 0.67407 


/Users/hbrent/PythonDS/MINE/myenv/lib/python3.7/site-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,


train - 0.69415 | valid - 0.67407 


/Users/hbrent/PythonDS/MINE/myenv/lib/python3.7/site-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,


...

/Users/hbrent/PythonDS/MINE/myenv/lib/python3.7/site-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,


train - 0.71476 | valid - 0.60741 


/Users/hbrent/PythonDS/MINE/myenv/lib/python3.7/site-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,


train - 0.68534 | valid - 0.63433 
train - 0.69769 | valid - 0.63433 
Average accuracy on crossval is  0.65798 
Std is 0.03532 
CPU times: user 24.2 s, sys: 7.62 s, total: 31.8 s
Wall time: 6.7 s


/Users/hbrent/PythonDS/MINE/myenv/lib/python3.7/site-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,


## 3. SVM regularization

### a. Default regularization

1. Train a baseline model with the only parameters `probability=True`, `kernel='linear'`, `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [10]:
model_svm = svm.SVC(probability=True, kernel='linear', random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(model_svm)

train - 0.71805 | valid - 0.70370 
train - 0.72877 | valid - 0.77037 
train - 0.76175 | valid - 0.71852 
train - 0.73207 | valid - 0.67407 
train - 0.72712 | valid - 0.65185 
train - 0.72547 | valid - 0.68889 
train - 0.74773 | valid - 0.76296 
train - 0.76257 | valid - 0.68889 
train - 0.77018 | valid - 0.70149 
train - 0.73888 | valid - 0.63433 
Average accuracy on crossval is  0.69951 
Std is 0.04102 


### b. Optimizing regularization parameters

1. In the cells below try different values of the parameter `C`.

In [ ]:
model_svm = svm.SVC(probability=True, kernel='linear', random_state=21, C = 10)
cv = StratifiedKFold(n_splits=10)
print_score(model_svm)

In [ ]:
model_svm = svm.SVC(probability=True, kernel='linear', random_state=21, C = 90)
cv = StratifiedKFold(n_splits=10)
print_score(model_svm)

## 4. Tree

### a. Default regularization

1. Train a baseline model with the only parameter `max_depth=10` and `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [ ]:
model_tree = DecisionTreeClassifier(max_depth=10, random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(model_tree)

### b. Optimizing regularization parameters

1. In the cells below try different values of the parameter `max_depth`.
2. As a bonus, play with other regularization parameters trying to find the best combination.

In [ ]:
model_tree = DecisionTreeClassifier(max_depth=2, random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(model_tree)

In [ ]:
model_tree = DecisionTreeClassifier(max_depth=6, random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(model_tree)

In [ ]:
model_tree = DecisionTreeClassifier(max_depth=14, random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(model_tree)

In [ ]:
model_tree = DecisionTreeClassifier(max_depth=18, random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(model_tree)

In [ ]:
model_tree = DecisionTreeClassifier(max_depth=22, random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(model_tree)

## 5. Random forest

### a. Default regularization

1. Train a baseline model with the only parameters `n_estimators=50`, `max_depth=14`, `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [ ]:
model_random_tree = RandomForestClassifier(n_estimators=50, max_depth=14, random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(model_random_tree)

### b. Optimizing regularization parameters

1. In the new cells try different values of the parameters `max_depth` and `n_estimators`.
2. As a bonus, play with other regularization parameters trying to find the best combination.

In [ ]:
model_random_tree = RandomForestClassifier(n_estimators=25, max_depth=14, random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(model_random_tree)

In [ ]:
model_random_tree = RandomForestClassifier(n_estimators=75, max_depth=14, random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(model_random_tree)

In [ ]:
model_random_tree = RandomForestClassifier(n_estimators=50, max_depth=7, random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(model_random_tree)

In [ ]:
model_random_tree = RandomForestClassifier(n_estimators=50, max_depth=21, random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(model_random_tree)

In [ ]:
model_random_tree = RandomForestClassifier(n_estimators=75, max_depth=21, random_state=21)
cv = StratifiedKFold(n_splits=10)
print_score(model_random_tree)

## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.
3. Analyze: for which weekday your model makes the most errors (in % of the total number of samples of that class in your test dataset).
4. Save the model.

In [ ]:
model_random_tree = RandomForestClassifier(n_estimators=75, max_depth=21, random_state=21)
cv = StratifiedKFold(n_splits=10)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_random_tree.fit(X_train, y_train)
y_pred=model_random_tree.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
plot_confusion_matrix(model_random_tree, X_test, y_test)
plt.show()

In [ ]:
filename = 'model_random_tree.sav'
pickle.dump(model_random_tree, open(filename, 'wb'))